In [1]:
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load data
file_path = Path("SQL_cleaning_resources/action_taken_summary.csv")
df = pd.read_csv(file_path)
df.head()

,as_of_year,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,action_taken_summary,...,purchaser_type,denial_reason_1,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,7/9/05,9,1,1,1,1,424,3,1,1.0,...,3,0,2,1,6585.0,70.16,75200.0,110.94,1352.0,1612.0
1,7/9/05,2,1,1,3,1,153,3,4,NaN,...,0,0,2,1,8152.0,22.46,67300.0,96.86,2404.0,3345.0
2,7/9/05,9,1,1,3,1,56,3,3,0.0,...,0,3,2,1,7388.0,4.24,65500.0,120.10,2651.0,2999.0
3,7/9/05,7,1,1,3,1,207,3,1,1.0,...,1,0,2,1,6993.0,3.15,87200.0,107.98,2168.0,2732.0
4,7/9/05,7,1,1,1,1,424,3,4,NaN,...,0,0,2,1,4775.0,67.64,51800.0,219.02,1305.0,1448.0


In [32]:
df.isnull().sum()

as_of_year                           0
agency_code                          0
loan_type                            0
property_type                        0
loan_purpose                         0
owner_occupancy                      0
loan_amount_000s                     0
preapproval                          0
action_taken                         0
action_taken_summary              3724
msamd                             2827
state_code                         325
county_code                        394
census_tract_number                457
applicant_ethnicity                  0
co_applicant_ethnicity               0
applicant_race_1                     0
co_applicant_race_1                  0
applicant_sex                        0
co_applicant_sex                     0
applicant_income_000s             3100
purchaser_type                       0
denial_reason_1                      0
hoepa_status                         0
lien_status                          0
population               

In [48]:
df2 = df.drop(columns=["action_taken", "as_of_year", "denial_reason_1", "msamd", "county_code", "census_tract_number"])
df2 = df2.dropna()
df2 = pd.get_dummies(df2, columns=)
df2.isnull().sum()

agency_code                       0
loan_type                         0
property_type                     0
loan_purpose                      0
owner_occupancy                   0
loan_amount_000s                  0
preapproval                       0
action_taken_summary              0
state_code                        0
applicant_ethnicity               0
co_applicant_ethnicity            0
applicant_race_1                  0
co_applicant_race_1               0
applicant_sex                     0
co_applicant_sex                  0
applicant_income_000s             0
purchaser_type                    0
hoepa_status                      0
lien_status                       0
population                        0
minority_population               0
hud_median_family_income          0
tract_to_msamd_income             0
number_of_owner_occupied_units    0
number_of_1_to_4_family_units     0
dtype: int64

In [49]:
# Define features and target
y = df2["action_taken_summary"]
X = df2.drop(columns="action_taken_summary")

In [50]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

In [51]:
# Creating StandardScaler instance, fitting to X, and scaling
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [52]:
# Creating the decision tree classifier instance and fitting to data
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

In [53]:
predictions = model.predict(X_test_scaled)

In [54]:
# Calculating confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index= ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

# Calculating accuracy score
acc_score = accuracy_score(y_test, predictions)

In [55]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score: {acc_score}")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,548,285
Actual 1,299,2878


Accuracy Score: 0.8543640897755611
              precision    recall  f1-score   support

         0.0       0.65      0.66      0.65       833
         1.0       0.91      0.91      0.91      3177

    accuracy                           0.85      4010
   macro avg       0.78      0.78      0.78      4010
weighted avg       0.86      0.85      0.85      4010



In [56]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=21)

In [57]:
# Fitting model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [58]:
# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

In [59]:
# Calculating confusion matrix
rf_cm = confusion_matrix(y_test, predictions)

rf_cm_df = pd.DataFrame(rf_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating accuracy score
rf_acc_score = accuracy_score(y_test, predictions)

In [60]:
# Displaying Results
print("Confusion Matrix")
display(rf_cm_df)
print(f"Accuracy Score : {rf_acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,548,285
Actual 1,299,2878


Accuracy Score : 0.8543640897755611
Classification Report
              precision    recall  f1-score   support

         0.0       0.65      0.66      0.65       833
         1.0       0.91      0.91      0.91      3177

    accuracy                           0.85      4010
   macro avg       0.78      0.78      0.78      4010
weighted avg       0.86      0.85      0.85      4010



In [61]:
# Sort features by importance
imp_list = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
imp_list

[(0.3787343875123473, 'purchaser_type'),
 (0.06387647922774545, 'applicant_income_000s'),
 (0.05771513098533533, 'agency_code'),
 (0.055718991522457385, 'loan_amount_000s'),
 (0.04743728116275096, 'tract_to_msamd_income'),
 (0.04591509697005611, 'minority_population'),
 (0.041853628188538786, 'population'),
 (0.04111769616900423, 'number_of_1_to_4_family_units'),
 (0.04110960231324314, 'number_of_owner_occupied_units'),
 (0.040533455593803724, 'hud_median_family_income'),
 (0.034173713953276605, 'state_code'),
 (0.026429577173197674, 'loan_purpose'),
 (0.024675116311286143, 'lien_status'),
 (0.019236720556391862, 'loan_type'),
 (0.013447817970150716, 'property_type'),
 (0.011543214033809717, 'applicant_race_1'),
 (0.010713354852242353, 'applicant_sex'),
 (0.010156984526834275, 'applicant_ethnicity'),
 (0.008462087957346863, 'co_applicant_race_1'),
 (0.007995491551439252, 'co_applicant_sex'),
 (0.0077876644331602105, 'co_applicant_ethnicity'),
 (0.00683985908022711, 'preapproval'),
 (0.